In [ ]:
import numpy as np

import sys
sys.path.insert(0, '..')

import matplotlib
import matplotlib.pyplot as plt
matplotlib.rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
matplotlib.rc('text', usetex=True)
matplotlib.rcParams.update({'font.size': 16})

from astropy.io import fits
from astropy.cosmology import Planck18 as cosmo
import astropy.units as u


from jpasLAEs.utils import bin_centers
from paus_utils import z_volume

In [ ]:
kokorev = fits.open('/home/alberto/almacen/LRD/VasilyKokorev-lrd_phot-a69c4c7/lrd_table_v1.1.fits')[1].data
selection = fits.open('/home/alberto/almacen/PAUS_data/catalogs/LAE_selection_with_MUV.fits')[1].data
print(kokorev.columns)

In [ ]:
from plot.puricomp1D import plot_puricomp1d

r_min, r_max = 17, 24
puricomp1d_L_bins = np.linspace(42.5, 45.5, 15)
puricomp1d_L_bins_c = bin_centers(puricomp1d_L_bins)
puri1d, comp1d = plot_puricomp1d('W1', 0, 18,
                                    17, 24,
                                    L_bins=puricomp1d_L_bins,
                                    LF_kind='Lya')

fig, ax = plt.subplots(figsize=(6, 4))

ax.plot(puricomp1d_L_bins_c, comp1d)
ax.plot(puricomp1d_L_bins_c, puri1d)

plt.show()

In [ ]:
z_min, z_max = 3, 4.5

sel_eff_vol = z_volume(z_min, z_max, 36)
k24_eff_vol = z_volume(z_min, z_max, 0.1773611111111111)

k24_z_mask = (kokorev.z_phot > z_min) & (kokorev.z_phot < z_max)
sel_z_mask = (selection['z_NB'] > z_min) & (selection['z_NB'] < z_max) & (selection['is_LAE_VI'])

print(sum(k24_z_mask))
print(sum(sel_z_mask))

In [ ]:
M_bins = np.linspace(-30, -15, 30)
M_bins_c = bin_centers(M_bins)
M_bin_w = M_bins[1] - M_bins[0]

k24_LF = np.histogram(kokorev.muv[k24_z_mask], M_bins)[0] / k24_eff_vol / M_bin_w
k24_LF_err = np.histogram(kokorev.muv[k24_z_mask], M_bins)[0]**0.5 / k24_eff_vol / M_bin_w

In [ ]:
# Correct sel_LF with puricomp1d
puri_sel = np.interp(selection['L_lya_corr'][sel_z_mask], puricomp1d_L_bins_c, puri1d)
comp_sel = np.interp(selection['L_lya_corr'][sel_z_mask], puricomp1d_L_bins_c, comp1d)
weights = 1. / comp_sel

sel_LF = np.histogram(selection['M_UV'][sel_z_mask], M_bins,
                      weights=weights)[0] / sel_eff_vol / M_bin_w
sel_LF_err = np.histogram(selection['M_UV'][sel_z_mask], M_bins,
                          weights=weights)[0]**0.5 / sel_eff_vol / M_bin_w

from jpasLAEs.utils import bin_centers
M_bins_c_sel = bin_centers(M_bins_c)
sel_LF = np.array([(sel_LF[i] + sel_LF[i + 1]) * 0.5 for i in range(len(M_bins_c_sel))])
sel_LF_err = (sel_LF  * sel_eff_vol * M_bin_w) ** 0.5 / sel_eff_vol / M_bin_w

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))

mask_LF = (k24_LF * k24_eff_vol * M_bin_w > 0)
ax.errorbar(M_bins_c[mask_LF], k24_LF[mask_LF], yerr=k24_LF_err[mask_LF],
            fmt='d', ls='')

mask_LF = (sel_LF * sel_eff_vol * M_bin_w > 0)
ax.errorbar(M_bins_c_sel[mask_LF], sel_LF[mask_LF], yerr=sel_LF_err[mask_LF],
            fmt='d', ls='')

ax.set_yscale('log')
ax.set_xlim(-15, -29)
ax.set_ylim(1e-10, 1e-3)

plt.show()